### Introduction to Parallel Programming - Debit Card
### A seriously non-deterministic program with a race condition
##### Adapted from https://realpython.com/intro-to-python-threading/

##### This logic spawns two threads, each makes a withdrawal from the same debit card acount.
##### The starting balance is 100. The two withdrawals are 16 and 14. 
##### The final balance should be (100 - 16 - 14), but it's not always correct due to a race condition.
##### Run this notebook and observe you will get different results for the final balance.

In [1]:
import multiprocessing as mp

import threading
import random 
from random import randint
from time import sleep

In [2]:
print("Number of processors (cores, actually): ", mp.cpu_count())

Number of processors (cores, actually):  12


In [3]:
# This is a global variable that is shared throughout this notebook.
# Threads are racing each other to access this variable.
global accountBalance   # Will be initialized in main
# Instantiate a random number generator that we will use to add some time delays. 
myRandom = random.Random()

In [4]:
### Everything falls apart in this logic if we run parallel threads
### We need to idenify the "Critical Section" and make it atomic. 
def makeWithdrawal(amount):
    '''
    amount = the $$$ to withdraw from the debit card account.
    We have introduced some random delays in the code to ensure non-determinstic results. 
    If we don't add delays, the code runs too fast, which is not a real-world scenario. 
    '''
    global accountBalance
    tmpAccountBalance = accountBalance
    sleep(myRandom.randint(0,2))
    tmpAccountBalance = tmpAccountBalance - amount
    sleep(myRandom.randint(0,2))
    accountBalance = tmpAccountBalance
    

In [5]:
def runInParallel():
    '''
    Make two withdrawals of 14 and 16 from the debit card account.
    The two withdrawals are in seperate threads, running in parallel.
    They will interfere with each other!
    '''
    global accountBalance
    print("Running parallelly... ")

    amazonPrimeThread = threading.Thread(target=makeWithdrawal, args=(16,))
    amazonPrimeThread.start()

    netFlixThread = threading.Thread(target=makeWithdrawal, args=(14,))
    netFlixThread.start()

    # Wait for all the threads to complete
    amazonPrimeThread.join()
    netFlixThread.join()


In [6]:
def runInSerial():
    global accountBalance
    print("Running serially... ")
    makeWithdrawal(16)
    makeWithdrawal(14)
    #print("Account balance = ", accountBalance)

In [7]:
def main():
    print("There's a race condition. This probably won't work the first time!")
    for attempts in range(1,1000):
        global accountBalance

        accountBalance = 100
        runInSerial()
        print("Final account balance = ", accountBalance)

        accountBalance = 100
        runInParallel()
        print("Final account balance = ", accountBalance)
        if accountBalance == 70:
            if attempts > 1:
                print("  running in parallel, it finally worked, after " , attempts, " attempts!")
            else:
                print("  running in parallel, it worked the first time!")
            break
        print("******************************************************")
        

In [8]:
if __name__ == '__main__':
    main()

There's a race condition. This probably won't work the first time!
Running serially... 
Final account balance =  70
Running parallelly... 
Final account balance =  84
******************************************************
Running serially... 
Final account balance =  70
Running parallelly... 
Final account balance =  84
******************************************************
Running serially... 
Final account balance =  70
Running parallelly... 
Final account balance =  84
******************************************************
Running serially... 
Final account balance =  70
Running parallelly... 
Final account balance =  86
******************************************************
Running serially... 
Final account balance =  70
Running parallelly... 
Final account balance =  86
******************************************************
Running serially... 
Final account balance =  70
Running parallelly... 
Final account balance =  86
******************************************************
Run